# <center> MNIST DIGIT PYTORCH CLASSIFIER </center>

### <center><span style='background :PowderBlue;width: 540px;clear:both;'> Results : Achieved 99.44% Accuracy on Test Data on 11th Epoch with Total Trainable Params: 19,834
</span></center>

### Process:

1. We Started with augmenting the training data. We applied Random Rotations, Scaling, Shearing transformations.
2. Created Train Loader and Test Loader
3. Designed the Neural Network using Convolution layers/Max Pool/Dropouts/GAP/Batch Normalization/FC with Total Trainable Params:19,834
4. Train and Test the Model


## Step 1 : Loading necessary packages:

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset, ConcatDataset

## Step 2 : Check for GPU 

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(1)
batch_size = 2056

## Step 3 : Data loading and Data Augmentation

### Train Loader

In [3]:

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    
    ConcatDataset([datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.RandomAffine(degrees=15, translate=(0.1,0.1), scale=(0.8,0.8)),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                  datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.RandomRotation(30),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                  datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                  datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.RandomAffine(degrees=30, scale=(1.1,1.1)),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                  datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.RandomAffine(degrees=10, shear=45),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))
                  
                  ]),batch_size=batch_size, shuffle=True, **kwargs)

### Test Loader

In [4]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

## Step 4: Training and Testing - Definitions

In [5]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [6]:
dropout_value = 0.125

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=24, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(24),
            nn.Dropout(dropout_value)
        ) # output_size = 26

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=24, out_channels=24, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(24),
            nn.Dropout(dropout_value)
        ) # output_size = 24

        # TRANSITION BLOCK 1
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=24, out_channels=22, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 24
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=22, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(20),
            nn.Dropout(dropout_value)
        ) # output_size = 10
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(20),
            nn.Dropout(dropout_value)
        ) # output_size = 8
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=18, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(18),
            # nn.Dropout(dropout_value)
        ) # output_size = 6
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=18, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),         
            nn.BatchNorm2d(16)
        ) # output_size = 6
        
        # OUTPUT BLOCK
        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=6)
        ) # output_size = 1

        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
        ) 
        self.dropout = nn.Dropout(dropout_value)
        self.ln = nn.Linear(10,10)


    def forward(self, img):
      img = self.convblock1(img)
      img = self.convblock2(img)
      img = self.convblock3(img)
      img = self.pool1(img)
      img = self.convblock4(img)
      img = self.convblock5(img)
      img = self.convblock6(img)
      img = self.convblock7(img)
      img = self.gap(img)  
      img = self.convblock8(img)      
      img = img.view(-1, 10)
      
      img = self.ln(img)
      
      return F.log_softmax(img, dim=-1)

In [7]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 26, 26]             216
              ReLU-2           [-1, 24, 26, 26]               0
       BatchNorm2d-3           [-1, 24, 26, 26]              48
           Dropout-4           [-1, 24, 26, 26]               0
            Conv2d-5           [-1, 24, 24, 24]           5,184
              ReLU-6           [-1, 24, 24, 24]               0
       BatchNorm2d-7           [-1, 24, 24, 24]              48
           Dropout-8           [-1, 24, 24, 24]               0
            Conv2d-9           [-1, 22, 24, 24]             528
        MaxPool2d-10           [-1, 22, 12, 12]               0
           Conv2d-11           [-1, 20, 10, 10]           3,960
             ReLU-12           [-1, 20, 10, 10]               0
      BatchNorm2d-13           [-1, 20, 10, 10]              40
          Dropout-14           [-1, 20,

## Step 5: Train & Test the Model

In [8]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0275, momentum=0.9)
EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|                                                                                          | 0/146 [00:00<?, ?it/s]

EPOCH: 0


Loss=0.15676774084568024 Batch_id=145 Accuracy=73.20: 100%|██████████████████████████| 146/146 [01:47<00:00,  1.36it/s]
  0%|                                                                                          | 0/146 [00:00<?, ?it/s]


Test set: Average loss: 0.0698, Accuracy: 9803/10000 (98.03%)

EPOCH: 1


Loss=0.08781187236309052 Batch_id=145 Accuracy=96.47: 100%|██████████████████████████| 146/146 [01:49<00:00,  1.34it/s]
  0%|                                                                                          | 0/146 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0177, Accuracy: 9944/10000 (99.44%)

EPOCH: 12


Loss=0.03963111713528633 Batch_id=145 Accuracy=98.62: 100%|██████████████████████████| 146/146 [01:47<00:00,  1.35it/s]
  0%|                                                                                          | 0/146 [00:00<?, ?it/s]


Test set: Average loss: 0.0189, Accuracy: 9939/10000 (99.39%)

EPOCH: 13


Loss=0.039776187390089035 Batch_id=87 Accuracy=98.60:  60%|████████████████▎          | 88/146 [01:07<00:44,  1.31it/s]


KeyboardInterrupt: 

# End